In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('HeartRateVariabilitySDNN.csv')

df['startDate'] = pd.to_datetime(df['startDate'])
df['endDate'] = pd.to_datetime(df['endDate'])
df = df.sort_values('startDate').reset_index(drop=True)


In [8]:
df.value.describe()

count    2993.000000
mean       28.527985
std        18.015771
min         4.115290
25%        17.327400
50%        24.115800
75%        33.707500
max       178.596000
Name: value, dtype: float64

In [24]:
# segment_duration_sec = 10
# start_time = df['startDate'].min()
# end_time = df['endDate'].max()

# current_start = start_time
# current_end = current_start + pd.Timedelta(seconds=segment_duration_sec)

# features_list = []

# while current_end <= end_time:
#     mask = (df['startDate'] < current_end) & (df['endDate'] > current_start)
#     segment = df.loc[mask]

#     if len(segment) >= 2:
#         values = segment['value'].values

#         hrv_sdnn = values.mean()

#         features_list.append({
#             'HRV_SDNN': hrv_sdnn,
#             'Segment_Start': current_start
#         })
#     current_start = current_end
#     current_end = current_start + pd.Timedelta(seconds=segment_duration_sec)

# feature_df = pd.DataFrame(features_list)

# print(feature_df.head())


In [25]:
df['HRV_SDNN'] = df['value']
df['HRV_SDNN_lag1'] = df['HRV_SDNN'].shift(1)
df['HRV_SDNN_lag2'] = df['HRV_SDNN'].shift(2)
cols = ['HRV_SDNN', 'HRV_SDNN_lag1', 'HRV_SDNN_lag2']

In [ ]:
window_size = 5
X = df[cols]
X = X.fillna(0)
X_sequences = []
for i in range(len(X) - window_size):        
    X_seq = X[i:i+window_size]
    X_sequences.append(X_seq)
X_sequences = np.array(X_sequences)


In [ ]:
from tensorflow.keras.models import load_model
model = load_model('/Users/tuoyuaner/Downloads/WESAD/lstm_emotion_model.h5')

y_pred_prob = model.predict(X_sequences)
y_pred = np.argmax(y_pred_prob, axis=1)

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2025-04-27 22:37:50.490763: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-27 22:37:50.490856: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [41]:
len(y_pred)

2988

In [44]:
pd.Series(y_pred).value_counts()

1    1821
0    1027
2     140
dtype: int64